# Ввод-вывод, файлы, директории

Откроем текстовый файл на чтение (когда второй аргумент не указан, файл открывается именно на чтение).

In [1]:
f=open('text.txt')
f,type(f)

(<_io.TextIOWrapper name='text.txt' mode='r' encoding='UTF-8'>,
 _io.TextIOWrapper)

Получился объект `f` одного из файловых типов. Что с ним можно делать? Можно его использовать в `for` цикле, каждый раз будет возвращаться очередная строка файла (включая `'\n'` в конце; в конце последней строки текстового файла `'\n'` может и не быть).

In [2]:
for s in f:
    print(s)

abcd

efgh

ijkl



Теперь файл нужно закрыть.

In [3]:
f.close()

Такой стиль работы с файлом (`f=open(...)`; работа с `f`; `f.close()`) на самом деле не рекомендуется. Гораздо правильнее использовать оператор `with`. Он гарантирует, что файл будет закрыт как в том случае, когда исполнение тела `with` нормально дошло до конца, так и тогда, когда при этом произошло исключение, и мы покинули тело `with` аварийно.

В операторе with может использоваться любой объект класса, реализующего методы `__enter__` и `__exit__`. Обычно это объект-файл, возвращаемый функцией `open`.

In [4]:
with open('text.txt') as f:
    for s in f:
        print(s[:-1])

abcd
efgh
ijkl


Метод `f.read(n)` читает `n` символов (когда файл близится к концу и прочитать именно `n` символов уже невозможно, читает меньше; в самый последний раз он читает 0 символов и возвращает `''`). Прочитаем файл по 1 символу.

In [5]:
with open('text.txt') as f:
    while True:
        c=f.read(1)
        if c=='':
            break
        else:
            print(c)

a
b
c
d


e
f
g
h


i
j
k
l




Вызов `f.read()` без аргумента читает файл целиком (что не очень разумно, если в нём много гигабайт).

In [6]:
with open('text.txt') as f:
    s=f.read()
s

'abcd\nefgh\nijkl\n'

`f.readline()` читает очередную строку (хотя проще использовать `for s in f:`).

In [7]:
with open('text.txt') as f:
    while True:
        s=f.readline()
        if s=='':
            break
        else:
            print(s)

abcd

efgh

ijkl



Метод `f.readlines()` возвращает список строк (опять же его лучше не применять для очень больших файлов).

In [8]:
with open('text.txt') as f:
    l=f.readlines()
l

['abcd\n', 'efgh\n', 'ijkl\n']

Теперь посмотрим, чем же оператор `with` лучше, чем пара `open` - `close`.

In [9]:
def a(name):
    global f
    f=open(name)
    s=f.readline()
    n=1/0
    f.close()
    return s

In [10]:
a('text.txt')

ZeroDivisionError: division by zero

In [11]:
f.closed

False

In [12]:
f.close()

Произошло исключение, мы покинули функцию до строчки `close`, и файл не закрылся.

In [13]:
def a(name):
    global f
    with open(name) as f:
        s=f.readline()
        n=1/0
    return s

In [14]:
a('text.txt')

ZeroDivisionError: division by zero

In [15]:
f.closed

True

Теперь всё в порядке.

Чтобы открыть файл на запись, нужно включить второй аргумент `'w'`.

In [16]:
f=open('newtext.txt','w')

In [17]:
f.write('aaa\n')

4

In [18]:
f.write('bbb\n')

4

In [19]:
f.write('ccc\n')

4

In [20]:
f.close()

Метод `write` возвращает число записанных символов.

Опять же, лучше использовать with.

In [21]:
with open('newtext.txt','w') as f:
    f.write('aaa\n')
    f.write('bbb\n')
    f.write('ccc\n')

In [22]:
!cat newtext.txt

aaa
bbb
ccc


Эта функция копирует старый текстовый файл в новый. Если строки нужно как-нибудь обработать, в последней строчке вместо `line` будет стоять что-нибудь вроде `f(line)`.

In [23]:
def copy(old_name,new_name):
    with open(old_name) as old,open(new_name,'w') as new:
        for line in old:
            new.write(line)

In [24]:
copy('text.txt','newtext.txt')

In [25]:
!cat newtext.txt

abcd
efgh
ijkl


В интерактивной сессии (или в программе, запущенной с командной строки) можно попросить пользователя что-нибудь ввести. Аргумент функции `input` - это приглашение для ввода (prompt). Можно использовать просто `input()`, тогда приглашения не будет. Но это неудобно, т.к. в этом случае трудно заметить, что программа чего-то ждёт.

In [26]:
s=input('Введите целое число ')

Введите целое число 123


In [27]:
s

'123'

In [28]:
n=int(s)
n

123

Питон - интерпретатор, поэтому он может во время выполнения программы интерпретировать строки как куски исходного текста на языке питон. Так, функция `eval` интерпретирует строку как выражение и вычисляет его (в текущем контексте - подставляя текущие значения переменных).

In [29]:
s=input('Введите выражение ')

Введите выражение n+1


In [30]:
s

'n+1'

In [31]:
eval(s)

124

А функция `exec` интерпретирует строку как оператор и выполняет его. Оператор может менять значения переменных в текущем пространстве имён.

In [32]:
s=input('Введите оператор ')

Введите оператор x=0


In [33]:
s

'x=0'

In [34]:
exec(s)
x

0

Строка `s` может быть результатом длинного и сложного вычисления. Но лучше таких фокусов не делать, так как программа фактически становится самомодифицирующейся. Такие программы очень сложно отлаживать.

Для работы с путями к файлам и директориям в стандартной библиотеке существует модуль `pathlib`. Объект класса `Path` представляет собой путь к файлу или директории.

In [35]:
from pathlib import Path

`Path()` возвращает текущую директорию.

In [36]:
p=Path()
p

PosixPath('.')

Очень полезный метод `resolve` приводит путь к каноническому виду.

In [37]:
p.resolve()

PosixPath('/home/grozin/python/book')

Путь может быть записан в совершенно идиотском виде; `resolve` его исправит.

In [38]:
p=Path('.././/book')
p=p.resolve()
p

PosixPath('/home/grozin/python/book')

Статический метод `cwd` возвращает текущую директорию (current working directory).

In [39]:
Path.cwd()

PosixPath('/home/grozin/python/book')

Если `p` - путь к директории, то можно посмотреть все файлы в ней.

In [40]:
for f in p.iterdir():
    print(f)

/home/grozin/python/book/b102_strings.ipynb
/home/grozin/python/book/tex
/home/grozin/python/book/.ipynb_checkpoints
/home/grozin/python/book/b103_lists.ipynb
/home/grozin/python/book/b109_exceptions.ipynb
/home/grozin/python/book/fac.py
/home/grozin/python/book/d1
/home/grozin/python/book/newtext.txt
/home/grozin/python/book/b108_oop.ipynb
/home/grozin/python/book/b106_dictionaries.ipynb
/home/grozin/python/book/b101_numbers.ipynb
/home/grozin/python/book/text.txt
/home/grozin/python/book/b104_tuples.ipynb
/home/grozin/python/book/__pycache__
/home/grozin/python/book/p1
/home/grozin/python/book/b107_functions.ipynb
/home/grozin/python/book/b110_modules.ipynb
/home/grozin/python/book/b105_sets.ipynb
/home/grozin/python/book/b111_input_output.ipynb


Если `p` - путь к директории, то `p/'fname'` - путь к файлу `fname` в ней (он, конечно, тоже может быть директорией).

In [41]:
p2=p/'b101_numbers.ipynb'
p2

PosixPath('/home/grozin/python/book/b101_numbers.ipynb')

Существует ли такой файл?

In [42]:
p2.exists()

True

Является ли он симлинком, директорией, файлом?

In [43]:
p2.is_symlink(),p2.is_dir(),p2.is_file()

(False, False, True)

Части пути `p2`.

In [44]:
p2.parts

('/', 'home', 'grozin', 'python', 'book', 'b101_numbers.ipynb')

Родитель - директория, в которой находится этот файл.

In [45]:
p2.parent,p2.parent.parent

(PosixPath('/home/grozin/python/book'), PosixPath('/home/grozin/python'))

Имя файла, его основа и суффикс.

In [46]:
p2.name,p2.stem,p2.suffix

('b101_numbers.ipynb', 'b101_numbers', '.ipynb')

Метод `stat` возвращает всякую ценную информацию о файле.

In [47]:
s=p2.stat()
s

os.stat_result(st_mode=33188, st_ino=2097706, st_dev=2052, st_nlink=1, st_uid=1000, st_gid=1000, st_size=17223, st_atime=1496721673, st_mtime=1496738332, st_ctime=1496799038)

Например, его размер в байтах.

In [48]:
s.st_size

17223

Я написал полезную утилиту для поиска одинаковых файлов. Ей передаётся произвольное число аргументов - директорий и файлов. Она рекурсивно обходит директории, находит размер всех файлов (симлинки игнорируются) и строит словарь, сопоставляющий каждому размеру список файлов, имеющих такой размер. Это простой этап, не требующий чтения (возможно больших) файлов. После этого файлы из тех списков, длина которых $>1$, сравниваются внешней программой `diff` (что, конечно, требует их чтения).

В питоне можно работать с переменными окружения как с обычным словарём.

In [49]:
from os import environ

In [50]:
environ['PATH']

'/usr/lib/python-exec/python3.6:/home/grozin/bin:/home/grozin/reduce-3783/bin:/usr/local/bin:/usr/bin:/bin:/opt/bin:/usr/games/bin'

In [51]:
environ['ABCD']

KeyError: 'ABCD'

In [52]:
environ['ABCD']='abcd'

In [53]:
environ['ABCD']

'abcd'

Мы не просто добавили пару ключ-значение в словарь, а действительно добавили новую переменную к текущему окружению. Если теперь вызвать из питона какую-нибудь внешнюю программу, то она эту переменную увидит. Эта переменная исчезнет, когда закончится выполнение текущей программы на питоне (или интерактивная сессия).